In [101]:
import pandas as pd
import os

In [122]:
def keyboard_data(eventFile):
#         if self.kbd_data is None:
#             print("Creating Keyboard Data CSV")
        df = event_data(eventFile)
        #df = self.event_data(graph_or_tree)
        labels = (df["Event"] != df["Event"].shift()).cumsum()
        df['wordGroup']= labels
        def concatInput(series):
            mask = (series.str.len()==1)
            return series.loc[mask].str.cat()
        groups = df[df["Event"] == 'Keyboard'].groupby(['wordGroup']).agg({'Time': ['min', 'max'], 'Description': concatInput})
        groups.columns = groups.columns.get_level_values(1)
        groups.filter(items=['min', 'max', 'concatInput'])
        groups.columns = ['startTime', 'endTime', 'typedString']

        kbd_data = groups

        #print(groups)
        return kbd_data

In [102]:
def event_data(eventFile):
#         if self.evd_data is None:
#             print("Creating Event Data CSV")
#         eventFile = self.filepath + "p1." + self.graph_or_tree + "EVD.txt"
    eventColNames = ['Time', 'Event', 'EventKey', 'Data1', 'Data2', 'Description']
    eventData = pd.read_csv(eventFile, delimiter="\t", error_bad_lines=False, names=eventColNames, header=None)
#             self.evd_data = eventData

    return eventData

In [103]:
def fixation_data(fixationFile):
#         if self.fxd_data is None:
#             print("Creating Fixation Data CSV")
#         fixationFile = self.filepath + "p1." + self.graph_or_tree + "FXD.txt"
    fixationColNames = ['ID', 'Time', 'Duration', 'X' , 'Y']
    fixationData = pd.read_csv(fixationFile, delimiter="\t", error_bad_lines=False, names=fixationColNames, header=None)
#             self.fxd_data = fixData
    
    
    return fixationData[((fixationData['X'] >0 ) & (fixationData['X'] < 1600)) & 
                        ((fixationData['Y'] >0 ) & (fixationData['Y'] < 1200))]

In [104]:
def baseline_data(baselineFile):
#         if self.gzd_data is None:
#             print("Creating Gaze Data CSV")
#         gazeFile = self.filepath + "p1." + self.graph_or_tree + "GZD.txt"
    baselineColNames = ['Time', 'ID', 'ScreenL X', 'ScreenL Y', 'CamL X', 'CamL Y', 'DistL', 'PupilL', 'CodeL',
                            'ScreenR X', 'ScreenR Y', 'CamR X', 'CamR Y', 'DistR', 'PupilR', 'CodeR']
#             gazeColNames = ["time", "number",
#                             "screenXR", "screenYR", "camXR", "camYR", "distanceR", "pupilR", "validityR",
#                             "screenXL", "screenYL", "camXL", "camYL", "distanceL", "pupilL", "validityL"
#                             ]
    baselineData = pd.read_csv(baselineFile, delimiter="\t", error_bad_lines=False, names=baselineColNames, header=None)
#             self.gzd_data = gazeData

    baselineData.drop(['ScreenL X', 'ScreenL Y', 'ScreenR X', 'ScreenR Y', 'CamL X', 'CamL Y', 'CamR X', 'CamR Y', 'DistL', 'DistR'], 
                 axis = 1, inplace = True)
    
    return baselineData

In [105]:
def gaze_data(gazeFile):
#         if self.gzd_data is None:
#             print("Creating Gaze Data CSV")
#             gazeFile = self.filepath + "p1." + self.graph_or_tree + "GZD.txt"
        gazeColNames = ['Time', 'ID', 'ScreenL X', 'ScreenL Y', 'CamL X', 'CamL Y', 'DistL', 'PupilL', 'CodeL',
                            'ScreenR X', 'ScreenR Y', 'CamR X', 'CamR Y', 'DistR', 'PupilR', 'CodeR']
#             gazeColNames = ["time", "number",
#                             "screenXR", "screenYR", "camXR", "camYR", "distanceR", "pupilR", "validityR",
#                             "screenXL", "screenYL", "camXL", "camYL", "distanceL", "pupilL", "validityL"
#                             ]
        gazeData = pd.read_csv(gazeFile, delimiter="\t", error_bad_lines=False, names=gazeColNames, header=None)
#             self.gzd_data = gazeData
#         return preprocess_gaze_data(gazeData,baselineData)

        gazeData.drop(['ScreenL X', 'ScreenL Y', 'ScreenR X', 'ScreenR Y', 'CamL X', 'CamL Y', 'CamR X', 'CamR Y', 'DistL', 'DistR'], 
                 axis = 1, inplace = True)
    
        return gazeData
        
#         return gazeData[gazeData['CodeL'] == 0 & gazeData['CodeR'] == 0 & 
#                         (gazeData['ScreenR X'] > 0 & gazeData['ScreenR X'] < 1600) &
#                         (gazeData['ScreenR Y'] > 0 & gazeData['ScreenR Y'] < 1200) &
#                         (gazeData['ScreenL X'] > 0 & gazeData['ScreenL X'] < 1600) &
#                         (gazeData['ScreenL Y'] > 0 & gazeData['ScreenL Y'] < 1200)]
        
#         return self.gzd_data.query('validityR <= 2 and validityL <= 2 '
#                                    'and screenXR > 0 and screenXR < 1600'
#                                    'and screenYR > 0 and screenYR < 1200'
#                                    'and screenXL > 0 and screenXL < 1600'
#                                    'and screenYL > 0 and screenYL < 1200')

In [163]:
participants = []
def create_dfs(archive_path):
    directories_table = {}
    for directory in os.listdir(archive_path):
        if directory != "__MACOSX" and directory != "Data":
            participants.append(directory)
            files_table = {}
            directories_table[directory] = files_table
            files = os.listdir(archive_path + directory)
            baselineFile = archive_path + directory + "\\" + files[-1]
            files_table[files[-1].replace('GZD','baseline')] = baseline_data(baselineFile)
#             print(type(baselineFile))
            for i in range(len(files)):
                if files[i] != ".DS_Store":
                    if "EVD" in files[i]:
                        eventFile = archive_path + directory + "\\" + files[i]
                        if "graphEVD" in files[i]:
                            files_table[files[i].replace('graphEVD','graphevent')] = event_data(eventFile)
                            files_table[directory+'graphKeyboardData'] = keyboard_data(eventFile)
                        if "tree" in files[i]:
                            files_table[files[i].replace('treeEVD','treeevent')] = event_data(eventFile)
                            files_table[directory+'treeKeyboardData'] = keyboard_data(eventFile)
#                         files_table[files[i]] = event_data(eventFile)
    #                     files_table[file] = pd.read_csv(filepath, 
    #                                                     sep='\t', 
    #                                                     error_bad_lines=False, 
    #                                                     names = ['Time', 'Event', 'EventKey', 'Data1', 'Data2', 'Description'] )
                    if "FXD" in files[i]:
                        fixationFile = archive_path + directory + "\\" + files[i]
                        if "graph" in files[i]:
                            files_table[files[i].replace('graphFXD','graphfixation')] = fixation_data(fixationFile)
                        if "tree" in files[i]:
                            files_table[files[i].replace('treeFXD','treefixation')] = fixation_data(fixationFile)
    #                   
#                         files_table[files[i]] = fixation_data(fixationFile)
    #                     files_table[file] = pd.read_csv(filepath, 
    #                                                     sep='\t', 
    #                                                     error_bad_lines=False, 
    #                                                     names = ['ID', 'Time', 'Duration', 'X' , 'Y'] )

                    if "GZD" in files[i]:
                        gazeFile = archive_path + directory + "\\" + files[i]
                        if "graph" in files[i]:
                            files_table[files[i].replace('graphGZD','graphgaze')] = gaze_data(gazeFile)
                        if "tree" in files[i]:
                            files_table[files[i].replace('treeGZD','treegaze')] = gaze_data(gazeFile)
    #                     files_table[file] = pd.read_csv(filepath, 
    #                                                     sep='\t', 
    #                                                     error_bad_lines=False, 
    #                                                     names = ['Time', 'ID', 'ScreenL X', 'ScreenL Y', 'CamL X', 'CamL Y', 'DistL', 'PupilL', 'CodeL',
    #                                                             'ScreenR X', 'ScreenR Y', 'CamR X', 'CamR Y', 'DistR', 'PupilR', 'CodeR'] )
                    
    return directories_table  

In [165]:
# directories_table = create_dfs("C:\\Users\\018094724sa\\Downloads\\Archive\\")
directories_table = create_dfs(r"C:\Users\adhee\Downloads\Archive\\")

In [261]:
for participant in participants:
#     print(directories_table[participant].keys())
    for key in directories_table[participant]:
        directories_table[participant][key]['Participant'] = participant

In [108]:
def preprocess_gaze_data(gazeData, baselineData):
    baselineData = baselineData.loc[(baselineData['CodeL'] == 0) & (baselineData['CodeR'] == 0)]
    
    gazeData = gazeData.loc[(gazeData['CodeL'] == 0) & (gazeData['CodeR'] == 0)]
    
    PupilLbaselineavg = baselineData['PupilL'].mean(axis = 0)
    PupilRbaselineavg = baselineData['PupilR'].mean(axis = 0)
    
    gazeData.loc[:,'PupilL'] = gazeData['PupilL'].apply(lambda x:x-PupilLbaselineavg)
    gazeData.loc[:,'PupilR'] = gazeData['PupilR'].apply(lambda x:x-PupilRbaselineavg)
    
    return gazeData
    
#     gazeData.drop(['ScreenL X', 'ScreenL Y', 'ScreenR X', 'ScreenR Y', 'CamL X', 'CamL Y', 'CodeL', 'CodeR', 'CamR X', 'CamR Y', 'PupilL', 'PupilR', 'DistL', 'DistR'], 
#                  axis = 1, inplace = True)

In [109]:
for participant in participants:
#     print(directories_table[participant].keys())
    for key in directories_table[participant]:
        if 'baseline' in key:
            baseline_key = key
#             preprocess_gaze_data(gazeData, baselineData)
        if 'graphgaze' in key:
            graphgaze_key = key
        if 'treegaze' in key:
            treegaze_key = key
    directories_table[participant][graphgaze_key] = preprocess_gaze_data(directories_table[participant][graphgaze_key], directories_table[participant][baseline_key])
    directories_table[participant][treegaze_key] = preprocess_gaze_data(directories_table[participant][treegaze_key], directories_table[participant][baseline_key])
    #             continue
        
#         if ''
#         print(key)

c:\python\python37\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [274]:
directories_table['p1']['p1.graphevent.txt'].head()

,Time,Event,EventKey,Data1,Data2,Description,Participant
0,11163,LMouseButton,1,237,610,NaN,p1
1,17931,LMouseButton,1,149,633,NaN,p1
2,25276,LMouseButton,1,1140,800,NaN,p1
3,27030,LMouseButton,1,1183,880,NaN,p1
4,28485,LMouseButton,1,1000,675,NaN,p1


In [264]:
def domain_segregation(archive_path):
    domain = {}
    file_path = archive_path + 'Additional Participant Data.csv'
    domain_df = pd.read_csv(file_path)
#   Ontologie -> 1 -> 'general'
#   Ontologie -> 2 -> 'expert'
#   Visualization -> 1 -> 'tree'
#   Visualization -> 2 -> 'graph'
#     for viz, ont in zip(domain_df['Visualization'], domain_df['Ontologies']):
#         if ont == 1 and viz == 1:
    participants = domain_df.loc[((domain_df['Visualization'] == 1) & (domain_df['Ontologies'] == 1))]['ID']
    files_table = {}
    for participant in participants:
        for key in directories_table[participant].keys():
            if "tree" in key:
                files_table[key] = directories_table[participant][key]
                domain['GeneralTree'] = files_table
                
    participants = domain_df.loc[((domain_df['Visualization'] == 1) & (domain_df['Ontologies'] == 2))]['ID']
    files_table = {}
    for participant in participants:
        for key in directories_table[participant].keys():
            if "tree" in key:
                files_table[key] = directories_table[participant][key]
                domain['ExpertTree'] = files_table
    
                
    participants = domain_df.loc[((domain_df['Visualization'] == 2) & (domain_df['Ontologies'] == 1))]['ID']
    files_table = {}
    for participant in participants:
        for key in directories_table[participant].keys():
            if "graph" in key:
                files_table[key] = directories_table[participant][key]
                domain['GeneralGraph'] = files_table
                
    participants = domain_df.loc[((domain_df['Visualization'] == 2) & (domain_df['Ontologies'] == 2))]['ID']
    files_table = {}
#     print(len(participants))
    for participant in participants:
        for key in directories_table[participant].keys():
            if "graph" in key:
                files_table[key] = directories_table[participant][key]
                domain['ExpertGraph'] = files_table
    return domain

In [265]:
# domain = domain_segregation("C:/Users/018094724sa/Downloads/Archive")
domain = domain_segregation("C:/Users/adhee/Downloads//")

In [266]:
domain['GeneralTree'].keys()

dict_keys(['p1.treeevent.txt', 'p1treeKeyboardData', 'p1.treefixation.txt', 'p1.treegaze.txt', 'p3.treeevent.txt', 'p3treeKeyboardData', 'p3.treefixation.txt', 'p3.treegaze.txt', 'p5.treeevent.txt', 'p5treeKeyboardData', 'p5.treefixation.txt', 'p5.treegaze.txt', 'p7.treeevent.txt', 'p7treeKeyboardData', 'p7.treefixation.txt', 'p7.treegaze.txt', 'p11.treeevent.txt', 'p11treeKeyboardData', 'p11.treefixation.txt', 'p11.treegaze.txt', 'p13.treeevent.txt', 'p13treeKeyboardData', 'p13.treefixation.txt', 'p13.treegaze.txt', 'p15.treeevent.txt', 'p15treeKeyboardData', 'p15.treefixation.txt', 'p15.treegaze.txt', 'p17.treeevent.txt', 'p17treeKeyboardData', 'p17.treefixation.txt', 'p17.treegaze.txt', 'p19.treeevent.txt', 'p19treeKeyboardData', 'p19.treefixation.txt', 'p19.treegaze.txt', 'p21.treeevent.txt', 'p21treeKeyboardData', 'p21.treefixation.txt', 'p21.treegaze.txt', 'p23.treeevent.txt', 'p23treeKeyboardData', 'p23.treefixation.txt', 'p23.treegaze.txt', 'p25.treeevent.txt', 'p25treeKeyboard

In [267]:
fixation_graph = {}
fixation_tree = {}

In [268]:
for category in domain.keys():
    categorized_graph_fixation_files = []
    categorized_tree_fixation_files = []
    for file in domain[category].keys():
        if 'fixation' in file:
#             print(file)
            if 'graph' in file:
                categorized_graph_fixation_files.append(domain[category][file])
            if 'tree' in file:
#                 print(domain[category][file].shape)
                categorized_tree_fixation_files.append(domain[category][file])
    if len(categorized_graph_fixation_files)!=0:
#         print(pd.concat(categorized_graph_fixation_files).reset_index(drop=True))
        fixation_graph[category] = pd.concat(categorized_graph_fixation_files).reset_index(drop=True)
    if len(categorized_tree_fixation_files)!=0:
#         print(pd.concat(categorized_tree_fixation_files).reset_index(drop=True))
        fixation_tree[category] = pd.concat(categorized_tree_fixation_files).reset_index(drop=True)

In [269]:
keyboard_graph = {}
keyboard_tree = {}

In [270]:
for category in domain.keys():
    categorized_graph_fixation_files = []
    categorized_tree_fixation_files = []
    for file in domain[category].keys():
        if 'Keyboard' in file:
#             print(file)
            if 'graph' in file:
                categorized_graph_fixation_files.append(domain[category][file])
            if 'tree' in file:
#                 print(domain[category][file].shape)
                categorized_tree_fixation_files.append(domain[category][file])
    if len(categorized_graph_fixation_files)!=0:
#         print(pd.concat(categorized_graph_fixation_files).reset_index(drop=True))
        keyboard_graph[category] = pd.concat(categorized_graph_fixation_files).reset_index(drop=True)
    if len(categorized_tree_fixation_files)!=0:
#         print(pd.concat(categorized_tree_fixation_files).reset_index(drop=True))
        keyboard_tree[category] = pd.concat(categorized_tree_fixation_files).reset_index(drop=True)

In [271]:
keyboard_graph['GeneralGraph']

,startTime,endTime,typedString,Participant
0,202702,204358,No,p4
1,503377,503766,yes,p4
2,583389,589741,{erPersonPerson,p4
3,593972,596911,EventEvent,p4
4,623890,627742,PrganOrg,p4
...,...,...,...,...
130,2225183,2225497,,p34
131,950681,953341,University,p36
132,956902,962334,University,p36
133,1237829,1242269,Conference,p36


In [272]:
for category in domain.keys():
    if 'Graph' in category:
        fixation_graph[category].to_csv('C:\\Users\\adhee\\Downloads\\Archive\Data\\'+category+'.csv')
    if 'Tree' in category:
        fixation_tree[category].to_csv('C:\\Users\\adhee\\Downloads\\Archive\Data\\'+category+'.csv')

In [144]:
graph = fixation_graph['ExpertGraph']

In [146]:
graph[graph['Duration'] <1200]

,ID,Time,Duration,X,Y
0,1,1104,100,728,396
1,2,2360,179,294,807
2,3,2858,279,281,777
3,4,3157,120,281,785
4,5,3297,319,275,792
...,...,...,...,...,...
68470,3758,2274877,100,494,276
68471,3759,2274997,359,497,235
68472,3760,2275376,279,494,272
68473,3761,2275675,199,486,243


In [117]:
import numpy as np
from PIL import Image
import math
from scipy.ndimage import gaussian_filter

In [118]:
def normalize(min, max, x):
    a = 13
    b = 255
    return (b-a) * (x - min)/(max - min) + a

In [119]:
def resize(filepath, graph_or_tree, size, img):
    img = img.resize(size)
    img.save(filepath + graph_or_tree + ".png", format='png')
    return img

In [149]:
"""
Create heatmap trace
"""
def create_heatmap(filepath, fxd_data, category, graph_or_tree):
    print("Creating Total Fixation Trace for "+ category)
    img = Image.open(filepath + graph_or_tree + ".png")
    img = resize(filepath , graph_or_tree, (1600, 1200) , img)
    img = img.convert('RGBA')
    pixels = np.array(img)

    zList = [[0 for x in range(2000)] for y in range(2000)]

    for index, row in fxd_data.iterrows():
        cx = row['X']
        cy = row['Y']
        z = row['Duration']
        r = 25
        for x in range(cx - r, cx + r):
            for y in range(cy - r, cy + r):
                d = math.sqrt((cx - x) ** 2 + (cy - y) ** 2)
                if d <= r and zList[y][x] < 1200:
                    zList[y][x] = zList[y][x] + float(z)

    blurred = gaussian_filter(zList, sigma = 25)


    min = 0
    max = np.amax(blurred)
    for i in range(len(pixels)):
        for j in range(len(pixels[0])):
                pixels[i][j][3] = normalize(min, max, blurred[i][j])

    img = Image.fromarray(pixels)
    img.save(filepath + "New" + category + ".png")

    trace.append(
        go.Heatmap(
            name = 'Total Fixation',
            z = blurred,
            type = 'heatmap',
            colorscale = ['rgba(0, 0, 0, 0.95)', 'rgba(0, 0, 0, 0.05)', 'rgba(0, 0, 0, 0)']
        )
    )


In [150]:
filepath = "C:\\Users\\adhee\\Downloads\\Archive\\Data\\"
for category in domain.keys():
    if 'Graph' in category: 
        create_heatmap(filepath, fixation_graph[category] , category, 'Graph')
    if 'Tree' in category:
        create_heatmap(filepath, fixation_tree[category] , category, 'Tree')

Creating Total Fixation Trace for GeneralTree
Creating Total Fixation Trace for ExpertTree


KeyboardInterrupt: 

In [313]:
def create_trace_fxd(fxd_data):
    print("Creating Slice Fixation Trace")
    end_time = max(fxd_data['Time'])

    # Saves a line-marker scatter plot for fixation between initial_time and initial_time + TIMESTEP
    # Increment initial_time by TIMESTEP
    # Repeat until end_time
    for initial_time in np.arange(0, end_time, TIMESTEP):
        final_time = initial_time + TIMESTEP
        fxd_subset = fxd_data[fxd_data['Time'].between(initial_time, final_time)]

        # start_time and end_time used for name of a scatterplot slice
        start = datetime.datetime.fromtimestamp(initial_time / 1000).strftime("%M:%S")
        end = datetime.datetime.fromtimestamp(final_time / 1000).strftime("%M:%S")
        fxd_subset['Text'] = "Duration: "+fxd_subset['Duration'].apply(lambda x: str(x)) +"ms "+"Participant: " + fxd_subset['Participant']
        trace.append(
            go.Scattergl(
                #name = start + '-' + end,
                name = 'Fixation Path',
                visible = False,
                text = fxd_subset['Text'],
                hoverinfo = 'text',
#                 hovertemplate = "<b>Duration: %{text}ms</b><extra></extra>",

                x = fxd_subset['X'],
                y = fxd_subset['Y'],
                mode = 'lines+markers',
                line = dict(
                    color ='red',
#                     color = '#FF69B4',
                    width = 2
                ),
                marker = dict(
                    color ='orange',
#                     color = 'lightgreen',
                    #color = '#FF69B4',
                    line = dict(
                        color ='orange',
#                         color = '#FF69B4',
                        width = 1
                    ),
                    size = fxd_subset['Duration']/4,
                    #size = 25
                )
            )
        )

In [ ]:
def create_trace_mcd(mcd_data):
    print("Creating Mouse Click Data Trace")
    # Saves a line-marker scatter plot for mouse data between initial_time and initial_time + TIMESTEP
    # Increment initial_time by TIMESTEP
    # Repeat until end_time
    end_time = max(mcd_data['time'])
    for initial_time in np.arange(0, end_time, TIMESTEP):
        final_time = initial_time + TIMESTEP
        mcd_subset = mcd_data[mcd_data['time'].between(initial_time, final_time)]

        trace.append(
            go.Scattergl(
                name = 'Mouse Clicks',
                visible=False,
                hoverinfo='none',
                x=mcd_subset['data1'],
                y=mcd_subset['data2'],
                mode='markers',
                marker=dict(
                    symbol = 'star-diamond-open-dot',
                    opacity=1.0,
                    line = dict(
                        width = 2
                    ),
                    #color='red',
                    color = 'red',
                    # color = 'black',
                    size=15,
                    # size = 25
                )
            )
        )

In [307]:
def create_sliders(fxd_data):
    print("Creating Sliders")
    steps = []
    end_time = max(fxd_data['Time'])
    data_len = end_time // TIMESTEP + (end_time % TIMESTEP > 0)

    # Each tick holds a boolean array of each trace. All traces are set to false in each tick.
    # Also sets graph title for each tick.
    # Also sets label, MM:SS, for each tick
    for i in range(data_len):
        timestamp = TIMESTEP * i / 1000
        step = dict(
            method='update',
            args=[{'visible': [False] * (2 * data_len)}
#                   {'title': create_trace_label(timestamp)}
                  ],
            label=datetime.datetime.fromtimestamp(timestamp).strftime("%M:%S"),
        )

        # Turns on the corresponding trace to each tick
        step['args'][0]['visible'][i] = True
        step['args'][0]['visible'][i + data_len] = True

        # Adds each tick to steps[]
        steps.append(step)

    # Creates sliders to hold steps[]
    sliders = [dict(
        #tickcolor = 'blue',
        minorticklen = 0,
        active=0,
        steps=steps,
        currentvalue={'prefix': 'Time: '},
        pad={'t': 50},
        font=dict(size=15)
    )]

    return sliders

In [305]:
def create_trace_label(timestamp):
    timeStr = datetime.datetime.fromtimestamp(timestamp).strftime("%M:%S") + " to " + datetime.datetime.fromtimestamp((timestamp + TIMESTEP / 1000)).strftime("%M:%S") + "<br>"
    periodStart = timestamp * 1000
    periodEnd = periodStart + TIMESTEP
#     row = keyboard_data.query('(@periodStart < startTime < @periodEnd) or @periodStart < endTime < @periodEnd').head(1)
    keyboardStr = ""
#     if row.empty:
#         nextRow = keyboard_data.query('startTime > @periodEnd').head(1)
#         if nextRow.empty:
#             keyboardStr = "No further typing"
#             color = 'grey'
#             #color = 'red'
#         else:
#             #print(nextRow)
#             keyboardStr = "before typing: " + nextRow.iloc[0]['typedString']
#             #color = 'blue'
#             color = 'red'
    else:
        #print(row)
        typingTimestamp = datetime.datetime.fromtimestamp(row['startTime'] // 1000).strftime("%M:%S")
        typingEnd = datetime.datetime.fromtimestamp(row['endTime'] // 1000).strftime("%M:%S")
        keyboardStr = "Typing: \"" + row.iloc[0]['typedString'] + "\" from " + typingTimestamp + " to " + typingEnd
        #color = 'orange'
        color = '#33a532'

    str = timeStr + " " + keyboardStr
    return go.layout.Title(
        text=str,
        xref='paper',
        x=0.5,
        y=0.97,
        font={
            'size': 36,
            'color': color
        },
        xanchor='center'
    )

In [323]:
def create_updatemenus():
    print("Creating Update Menus")
    
    fixation_off = [go.layout.Image(
                source = Image.open(filepath + 'Tree' + ".png"),
                opacity = 1.0,
                xref = 'x',
                yref = 'y',
                x = 0,
                y = 0,
                sizex = 1600,
                sizey = 1200,
                sizing = 'stretch',
                layer='below'
            )]

    fixation_on = [go.layout.Image(
                source = Image.open(filepath + "New" + 'ExpertTree' + ".png"),
                opacity = 1.0,
                xref = 'x',
                yref = 'y',
                x = 0,
                y = 0,
                sizex = 1600,
                sizey = 1200,
                sizing = 'stretch',
                layer='below'
            ), 
            create_trace_fxd(fixation_tree['ExpertTree'])]

    updatemenus = [
        go.layout.Updatemenu(
            x = 1.26,
            type = 'buttons',
            font=dict(size=30),
            buttons=[
                dict(label='Total Fixation On',
                     method='relayout',
                     args=['images', fixation_on]),
                dict(label='Total Fixation Off',
                     method='relayout',
                     args=['images', fixation_off]),
            ]
        )
    ]

    return updatemenus

In [217]:
from PIL import Image
import plotly.graph_objects as go
import plotly.offline as py

"""
Class to hold a figure for data
"""
class CoverageFigure:
    def __init__(self):
        self.figure = go.Figure()

    """
    Initializes layout of figure
    """
    def initialize_layout(self, img):
        self.figure.update_layout(
            title = go.layout.Title(
                text = 'Fixation and Events',
                xref = 'paper',
                x = 0.5,
                font = dict(
                    size=36,
                    color = 'black'
                ),
                xanchor = 'center'
            ),
            width = 1600,
            height = 1200,

            legend=dict(
                y = 0.91,
                font = dict(
                    size = 30
                )
            ),

            hoverlabel = dict(
                bgcolor = 'pink',
                font = dict(
                    size = 15
                )
            ),

            images = [go.layout.Image(
                source = Image.open(img),
                opacity = 1.0,
                xref = 'x',
                yref = 'y',
                x = 0,
                y = 0,
                sizex = 1600,
                sizey = 1200,
                sizing = 'stretch',
                layer='below'
            )],

            shapes=[
                go.layout.Shape(
                    type="rect",
                    x0=0,
                    y0=0,
                    x1=1600,
                    y1=1200,
                    fillcolor='black',
                    layer='below'
                )
            ],

            xaxis=dict(
                showline=True,
                showgrid = False,
                mirror=True,
                #tickfont = dict(size = 30),
                linewidth=2,
                linecolor='black',
                #gridcolor='#000000',
                range=[0, 1600],
                tickvals = []
                #tickvals=[k*200+100 for k in range(0, 8)],
                #ticktext=['1', '2', '3', '4', '5', '6', '7', '8', '9']

            ),
            yaxis=dict(
                showline=True,
                showgrid = False,
                mirror=True,
                #tickfont = dict(size = 30),
                linewidth=2,
                linecolor='black',
                #gridcolor='#000000',
                range=[1200, 0],
                tickvals = []
                #tickvals=[k*200+100 for k in range(0, 6)],
                #ticktext=['A', 'B', 'C', 'D', 'E', 'F', 'G']
            )
        )

    """
    Updates figure data/traces
    """
    def update_data(self, data):
        self.figure.data = None

        for trace in data:
            self.figure.add_trace(trace)

        self.figure.data[0].visible = True
        self.figure.data[int(len(data)/2)].visible = True

    """
    Updates figure layout with sliders
    """
    def update_layout(self, sliders, updatemenus):
        self.figure.update_layout(
            sliders=sliders,
            updatemenus = updatemenus,
            showlegend = True,
        )
#     def update_layout(self,updatemenus):
#         self.figure.update_layout(
#             updatemenus = updatemenus,
#             showlegend = True,
#         )

    """
    Saves figure as html
    """
    def show(self, file_path, file_name):
        py.plot(self.figure, filename=file_path + file_name + '.html')

In [324]:
TIMESTEP = 10000
import datetime
# WIDTH = 1600
# HEIGHT = 1200

# FILEPATH = "Data/"
# FILENAME = "graph"

# data = PreprocessData(FILEPATH, FILENAME)

# create trace
trace = []
#create_heatmap(data.fixation_data())
# create_trace_fxd(data.fixation_data())
# create_trace_mcd(data.mouse_data())

# create sliders
sliders = create_sliders(fixation_tree['ExpertTree'])

# create updatemenus
filepath = "C:\\Users\\adhee\\Downloads\\Archive\\Data\\"
updatemenus = create_updatemenus()

# create figure, update data with trace, update layout with sliders
figure = CoverageFigure()
figure.initialize_layout("C:\\Users\\adhee\\Downloads\\Archive\\Data\\" + "New" + "ExpertTree" + ".png")
figure.update_data(trace)
figure.update_layout(sliders, updatemenus)
# figure.update_layout(updatemenus)
figure.show("C:\\Users\\adhee\\Downloads\\Archive\\Data\\","outputExpertTree")

Creating Sliders
Creating Update Menus
Creating Slice Fixation Trace


c:\python\python37\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

